# Extending JWST PSFs

This tutorial demonstrates how to build extended JWST ePSF grids using `mophongo.jwst_psf`. It converts a directory of standard PSF grids into extended versions suitable for modelling wide halos.

In [1]:
from pathlib import Path
import os
import logging
from photutils.psf import STDPSFGrid 
from mophongo import jwst_psf
logging.basicConfig(level=logging.WARNING)

/Users/ivo/Astro/PROJECTS/MOPHONGO/mophongo/mophongo/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO: NumExpr defaulting to 10 threads.


## create several ST / Webb PSFs for every detector listed 

* search wcs_csv rate files produced by grizli to see which detectors
* do load_wss_opd_by_date closest to modal observation data of the mosaic 
* produce 4x oversampled (OS4) and detector sampled (DET) 
* produce single PSFs (center of detector) and square sampled grids (3x3 in 770, 5x5 in 444)



In [6]:
from pathlib import Path
from mophongo.jwst_psf  import psf_grid_from_csv
import glob

data_dir = Path('../data')
psf_dir = data_dir / 'PSF'
filters =  [('f770w', 8.0, [1,9]),
            ('f1280w', 8.0, [1,9]),
            ('f1500w', 8.0, [1,9]),
            ('f1800w', 8.0, [1,9])]

pf = 'UDS'
for filt, fov, ngrid in filters:
    print(filt)
    csv = glob.glob(str(data_dir)+f'/uds*{filt}*_wcs.csv')[0]
    #csv = data_dir / f'uds-test-{filt}_wcs.csv'
    for size in ngrid:
        print(csv,filt, fov, size)
        psf_grid_from_csv(csv, fov_arcsec=fov, num_psfs=size, prefix=pf, outdir=psf_dir,
                          postfix=f'DET_GRID{size}', use_detsampled_psf=True, save=True)
        psf_grid_from_csv(csv, fov_arcsec=fov, num_psfs=size, prefix=pf, outdir=psf_dir,
                          postfix=f'OS4_GRID{size}',save=True)


INFO: MIRI SIAF aperture name updated to MIRIM_FULL


f770w
../data/uds-lowres-all-f770w_drz_wcs.csv f770w 8.0 1
OUTFILE: UDS_MIRI_F770W_DET_GRID1.fits
Skipping UDS_MIRI_F770W_DET_GRID1.fits (exists, overwrite=False)
OUTFILE: UDS_MIRI_F770W_OS4_GRID1.fits
Skipping UDS_MIRI_F770W_OS4_GRID1.fits (exists, overwrite=False)
../data/uds-lowres-all-f770w_drz_wcs.csv f770w 8.0 9
OUTFILE: UDS_MIRI_F770W_DET_GRID9.fits
Skipping UDS_MIRI_F770W_DET_GRID9.fits (exists, overwrite=False)
OUTFILE: UDS_MIRI_F770W_OS4_GRID9.fits
Skipping UDS_MIRI_F770W_OS4_GRID9.fits (exists, overwrite=False)
f1280w
../data/uds-v2.3_f1280w_wcs.csv f1280w 8.0 1
OUTFILE: UDS_MIRI_F1280W_DET_GRID1.fits
iterating query, tdelta=3.0

MAST OPD query around UTC: 60883.82292009259
                        MJD: 60883.82292009259

OPD immediately preceding the given datetime:
	URI:	 mast:JWST/product/R2025072502-NRCA1_FP6-1.fits
	Date (MJD):	 60881.2737
	Delta time:	 -2.5492 days

OPD immediately following the given datetime:
	URI:	 mast:JWST/product/R2025072902-NRCA1_FP6-1.fits
	Date

INFO: NIRCam aperture name updated to NRCA1_FULL
INFO: NIRCam aperture name updated to NRCA1_FP6
INFO: OPD from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/NIRCam/OPD/wss_target_phase_fp6.fits: Loaded OPD from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/NIRCam/OPD/wss_target_phase_fp6.fits
INFO: No info supplied on amplitude transmission; assuming uniform throughput = 1
INFO: Creating optical system model:
INFO: Initialized OpticalSystem: JWST+NIRCam
INFO: JWST Entrance Pupil: Loaded amplitude transmission from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/jwst_pupil_RevW_npix1024.fits.gz
INFO: JWST Entrance Pupil: Loaded OPD from supplied fits.HDUList object
INFO: Loading field dependent model parameters from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/NIRCam/OPD/field_dep_table_nircam.fits
INFO: Calculating field-dependent OTE OPD at v2 = 1.591 arcmin, v3 = -8.384 arcmin
INFO: Added pupil plane: JWST Entrance Pupil
INFO: Added coordinate inversion plane: OTE exit pupil
INFO: Ad

Wrote 1 PSFs ➜ ../data/PSF/UDS_MIRI_F1280W_DET_GRID1.fits
OUTFILE: UDS_MIRI_F1280W_OS4_GRID1.fits
iterating query, tdelta=3.0


INFO: NIRCam aperture name updated to NRCA1_FULL
INFO: NIRCam aperture name updated to NRCA1_FP6
INFO: OPD from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/NIRCam/OPD/wss_target_phase_fp6.fits: Loaded OPD from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/NIRCam/OPD/wss_target_phase_fp6.fits
INFO: No info supplied on amplitude transmission; assuming uniform throughput = 1
INFO: Creating optical system model:
INFO: Initialized OpticalSystem: JWST+NIRCam
INFO: JWST Entrance Pupil: Loaded amplitude transmission from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/jwst_pupil_RevW_npix1024.fits.gz
INFO: JWST Entrance Pupil: Loaded OPD from supplied fits.HDUList object
INFO: Loading field dependent model parameters from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/NIRCam/OPD/field_dep_table_nircam.fits
INFO: Calculating field-dependent OTE OPD at v2 = 1.591 arcmin, v3 = -8.384 arcmin
INFO: Added pupil plane: JWST Entrance Pupil
INFO: Added coordinate inversion plane: OTE exit pupil
INFO: Ad

...

Wrote 1 PSFs ➜ ../data/PSF/UDS_MIRI_F1800W_OS4_GRID1.fits
../data/uds-v2.3_f1800w_wcs.csv f1800w 8.0 9
OUTFILE: UDS_MIRI_F1800W_DET_GRID9.fits

MAST OPD query around UTC: 59967.17310263056
                        MJD: 59967.17310263056

OPD immediately preceding the given datetime:
	URI:	 mast:JWST/product/R2023012202-NRCA3_FP1-1.fits
	Date (MJD):	 59965.8570
	Delta time:	 -1.3161 days

OPD immediately following the given datetime:
	URI:	 mast:JWST/product/R2023012303-NRCA3_FP1-1.fits
	Date (MJD):	 59967.6511
	Delta time:	 0.4780 days
User requested choosing OPD time closest in time to 59967.17310263056, which is R2023012303-NRCA3_FP1-1.fits, delta time 0.478 days
Importing and format-converting OPD from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/MAST_JWST_WSS_OPDs/R2023012303-NRCA3_FP1-1.fits
Backing out SI WFE and OTE field dependence at the WF sensing field point (NRCA3_FP1)


INFO: Added pupil plane: JWST Entrance Pupil
INFO: Added coordinate inversion plane: OTE exit pupil
INFO: Added pupil plane: NIRCamSWA internal WFE at V2V3=(1.16,-8.60)', near MIMF5
INFO: Added detector with pixelscale=0.031218915 and oversampling=2: NIRCam detector
INFO: Calculating field-dependent OTE OPD at v2 = 1.160 arcmin, v3 = -8.597 arcmin
INFO: No source spectrum supplied, therefore defaulting to 5700 K blackbody
INFO: Computing wavelength weights using synthetic photometry for F1800W...
INFO: PSF calc using fov_arcsec = 8.000000, oversample = 4, number of wavelengths = 9
INFO: Creating optical system model:
INFO: Initialized OpticalSystem: JWST+MIRI
INFO: JWST Entrance Pupil: Loaded amplitude transmission from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/jwst_pupil_RevW_npix1024.fits.gz
INFO: JWST Entrance Pupil: Loaded OPD from supplied fits.HDUList object
INFO: Loading field dependent model parameters from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/MIRI/OPD/field_dep_ta

Wrote 9 PSFs ➜ ../data/PSF/UDS_MIRI_F1800W_DET_GRID9.fits
OUTFILE: UDS_MIRI_F1800W_OS4_GRID9.fits

MAST OPD query around UTC: 59967.17310263056
                        MJD: 59967.17310263056

OPD immediately preceding the given datetime:
	URI:	 mast:JWST/product/R2023012202-NRCA3_FP1-1.fits
	Date (MJD):	 59965.8570
	Delta time:	 -1.3161 days

OPD immediately following the given datetime:
	URI:	 mast:JWST/product/R2023012303-NRCA3_FP1-1.fits
	Date (MJD):	 59967.6511
	Delta time:	 0.4780 days
User requested choosing OPD time closest in time to 59967.17310263056, which is R2023012303-NRCA3_FP1-1.fits, delta time 0.478 days
Importing and format-converting OPD from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/MAST_JWST_WSS_OPDs/R2023012303-NRCA3_FP1-1.fits
Backing out SI WFE and OTE field dependence at the WF sensing field point (NRCA3_FP1)


INFO: Added pupil plane: JWST Entrance Pupil
INFO: Added coordinate inversion plane: OTE exit pupil
INFO: Added pupil plane: NIRCamSWA internal WFE at V2V3=(1.16,-8.60)', near MIMF5
INFO: Added detector with pixelscale=0.031218915 and oversampling=2: NIRCam detector
INFO: Calculating field-dependent OTE OPD at v2 = 1.160 arcmin, v3 = -8.597 arcmin
INFO: No source spectrum supplied, therefore defaulting to 5700 K blackbody
INFO: Computing wavelength weights using synthetic photometry for F1800W...
INFO: PSF calc using fov_arcsec = 8.000000, oversample = 4, number of wavelengths = 9
INFO: Creating optical system model:
INFO: Initialized OpticalSystem: JWST+MIRI
INFO: JWST Entrance Pupil: Loaded amplitude transmission from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/jwst_pupil_RevW_npix1024.fits.gz
INFO: JWST Entrance Pupil: Loaded OPD from supplied fits.HDUList object
INFO: Loading field dependent model parameters from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/MIRI/OPD/field_dep_ta

Wrote 9 PSFs ➜ ../data/PSF/UDS_MIRI_F1800W_OS4_GRID9.fits


### Oversample DETECTOR sampled PSFs 

So that it remains compatible with DrizzlePSF which has x4 oversampling hard coded

In [7]:
from pathlib import Path
from astropy.io import fits
from scipy.ndimage import zoom
import numpy as np

psf_dir = Path('../data/PSF')
oversample_factor = 4

for f in psf_dir.glob('*DET*GRID*.fits'):
    outfile = f.with_name(f.stem + f'_OS{oversample_factor}.fits')
    with fits.open(f) as hdul:
        data = hdul[0].data  # shape (Npsf, Y, X)
        header = hdul[0].header.copy()
    if data.ndim == 2:
        data = data[np.newaxis, ...]
    oversampled = zoom(data, (1, oversample_factor, oversample_factor), order=3)
    fits.writeto(outfile, oversampled, header, overwrite=True)
    print(f"Oversampled PSF saved to {outfile} with shape {oversampled.shape}")


Oversampled PSF saved to ../data/PSF/UDS_NRCA5_F444W_DET_GRID25_OS4.fits with shape (25, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCA4_F115W_DET_GRID25_OS4_OS4.fits with shape (25, 1040, 1040)
Oversampled PSF saved to ../data/PSF/UDS_NRCB5_F444W_DET_GRID1_OS4_OS4.fits with shape (1, 1040, 1040)
Oversampled PSF saved to ../data/PSF/UDS_NRCA3_F115W_DET_GRID1_OS4.fits with shape (1, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCB4_F115W_DET_GRID1_OS4.fits with shape (1, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_MIRI_F1800W_DET_GRID1_OS4.fits with shape (1, 292, 292)
Oversampled PSF saved to ../data/PSF/UDS_NRCA3_F115W_DET_GRID25_OS4_OS4.fits with shape (25, 1040, 1040)
Oversampled PSF saved to ../data/PSF/UDS_NRCA5_F444W_DET_GRID25_OS4_OS4.fits with shape (25, 1040, 1040)
Oversampled PSF saved to ../data/PSF/UDS_NRCB1_F115W_DET_GRID1_OS4.fits with shape (1, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCB3_F115W_DET_GRID1_OS4_OS4.fits with shape (1, 1040

## Extending existing empirical ePSFs with wings from STPSF

Specify the directory containing the JWST STDPSF `*.fits` files. The extended PSFs will be written EXTENDED postfix


## Extending existing empirical ePSFs with wings from STPSF

Alternative approach. Use ePSF as core, extend with STPSF. The extended PSFs will be written EXTENDED postfix


In [ ]:

psf_dir = Path('../data/PSF')
outdir = psf_dir 
for f in psf_dir.rglob('*.fits'):
    outname = outdir / f.name.replace('.fits', '_EXTENDED.fits')  # extended psf
    outname_stpsf = outdir / f.name.replace('.fits', '_WEBB.fits') # webb stpsf

    if outname.exists() or 'EXTENDED' in f.name or 'WEBB' in f.name \
        or 'NRC' not in f.name or not '444' in f.name:
        continue

    epsf = STDPSFGrid(str(f))
    epsf_ext, st_psf = jwst_psf.make_extended_grid(epsf, Rmax=2.0, Rtaper=0.2, bg_pct=10, return_stpsf=True, verbose=True)

    os.makedirs(outdir, exist_ok=True)
    if not outname.exists(): 
        jwst_psf.write_stdpsf(outname, epsf_ext, overwrite=True, verbose=True)
    if not outname_stpsf.exists(): 
        jwst_psf.write_stdpsf(outname_stpsf, st_psf, overwrite=True, verbose=True)

Oversampled PSF saved to ../data/PSF/UDS_NRCA5_F444W_DET_GRID25_OS4.fits with shape (25, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCA3_F115W_DET_GRID1_OS4.fits with shape (1, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCB4_F115W_DET_GRID1_OS4.fits with shape (1, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCB1_F115W_DET_GRID1_OS4.fits with shape (1, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCB5_F444W_DET_GRID25_OS4.fits with shape (25, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCA1_F115W_DET_GRID25_OS4.fits with shape (25, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCB3_F115W_DET_GRID25_OS4.fits with shape (25, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCA3_F115W_DET_GRID25_OS4.fits with shape (25, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCB5_F444W_DET_GRID1_OS4.fits with shape (1, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_MIRI_F770W_DET_GRID9_OS4.fits with shape (9, 292, 292)
Oversampled PSF saved